Importación de librerías

In [7]:
import pandas as pd
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from numpy.lib.npyio import savez_compressed
#!/usr/bin/env python
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import normalize, to_categorical
from IPython.display import display
from sklearn.model_selection import train_test_split
from random import uniform
from scipy.sparse import random
from numpy import random as np_random

In [ ]:
#carga datos
#data = pd.read_csv('lp5.csv', sep=',')
from google.colab import files
uploaded = files.upload()
data = np.loadtxt('lp5.csv', delimiter=',',dtype=str)
data

In [ ]:
#Se separan las categorias y las matrices en listas separadas
count = 0
matriz = []
label = []
for i in range(2624): 
    if (count==0):
        label.append((data[i]))
    if (count > 0):
        matriz.append((data[i]))
    count+=1 
    if (count == 16): 
      count = 0

In [ ]:
#se anaden los valores a una matriz
matriz_data=np.zeros((len(matriz), 6))

for i in range(2460):
    for j in range(6):
        matriz_data[i][j] = float(matriz[i][j])
display(matriz_data)

#normalizar datos

# utilizando metodo de min max
valor_minimo = np.min(matriz_data)
valor_maximo = np.max(matriz_data)
matriz_data_Nor = matriz_data - valor_minimo
matriz_data_Nor =matriz_data_Nor / (valor_maximo - valor_minimo)

display ("matriz de datos", matriz_data_Nor)

In [ ]:
#Se genera la matriz de 3 dimenciones necesaria para el modulo convolucional
count = 0
mat_final=np.zeros((164,15,6))
for i in range(164): 
      for j in range(15):
          for k in range(6):
              mat_final[i][j][k] = matriz_data_Nor[count][k] 
          count += 1 

mat_final

In [ ]:
#Se genera la lista con la categorixacion de las labels.
cat = pd.DataFrame(label, columns=['category', '1', '2', '3', '4', '5'])
if any(cat['category'] == 'normal'):
    cat.loc[cat['category'] == 'normal' ] = ['normal',1, 0, 0, 0, 0]
if any(cat['category'] == 'collision_in_tool'):
    cat.loc[cat['category'] == 'collision_in_tool' ] = ['collision_in_tool',0, 1, 0, 0, 0]
if any(cat['category'] == 'collision_in_part'):
    cat.loc[cat['category'] == 'collision_in_part' ] = ['collision_in_part', 0, 0, 1, 0, 0]
if any(cat['category'] == 'bottom_collision'):
    cat.loc[cat['category'] == 'bottom_collision' ] = ['bottom_collision',0, 0, 0, 1, 0]
if any(cat['category'] == 'bottom_obstruction'):
    cat.loc[cat['category'] == 'bottom_obstruction' ] = ['bottom_collision',0, 0, 0, 0, 1]
clases_clasificacion1 =cat.drop(['category'], axis=1)
clases_clasificacion= clases_clasificacion1.astype(int)

In [ ]:
#Division de datos de entrenamiento
X_train,X_test,y_train,y_test = train_test_split( mat_final, clases_clasificacion, test_size=0.4, random_state=42)
X_train = np.array([np.array(val) for val in X_train])


#Division de datos para validacion para validacion 1
X_validacion,X_test,y_validacion,y_test = train_test_split( X_test, y_test, test_size=0.4, random_state=42)

#obtencion de datos sinteticos para validacin 2
display(np.shape(X_test))

ruido_sintetico=np.zeros((27,15,6))
for i in range(27): 
      for j in range(15):
          for k in range(6):
              b=np_random.uniform(-0.05,0.05)
              ruido_sintetico[i][j][k] =X_test[i][j][k]+ ((X_test[i][j][k] )*b)


In [ ]:
# Capas del modelo convolucional
activation = 'relu'
model = Sequential()
model.add(Conv2D(32, kernel_size=3, activation = activation, strides=(2, 1), input_shape=(15,6,1) ))

model.add(Conv2D(64, kernel_size=3,strides=(1,1),  padding = 'same', activation = activation))
model.add(MaxPooling2D())

model.add(Conv2D(64, kernel_size=3,strides=(1, 1), padding = 'same', activation = activation )) 
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(16, activation = activation))
model.add(Dense(5, activation = 'softmax'))


In [ ]:
#Definicion del modelo y ejecucion de entrenamiento
model.compile(optimizer = 'rmsprop',loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=200, batch_size=4,validation_data =(X_test, y_test))

from sklearn.metrics import confusion_matrix
model.predict(ruido_sintetico)
#predictions= model.predict(ruido_sintetico)
#rounded_labels=np.argmax(predictions, axis=1)#
#Y_real=np.argmax(y_test, axis=1)
#cm = confusion_matrix(Y_real, rounded_labels)

model.predict(X_validacion)
#predictions= model.predict(X_validacion)
#rounded_labels=np.argmax(predictions, axis=1)#
#Y_real=np.argmax(y_test, axis=1)
#cm = confusion_matrix(Y_real, rounded_labels)